# Validation comparing queried articles with random articles

### Sagar Setru, September 21th, 2020

In [1]:
# make sure I'm in the right environment (should be 'debiaser')
import os
print('Conda environment:')
print(os.environ['CONDA_DEFAULT_ENV'])

Conda environment:
debiaser


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

import json

# NLP Packages
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# to break articles up into sentences
from nltk import tokenize

import pyLDAvis
import pyLDAvis.gensim

import pickle

from debiaser_validation_function import return_suggested_articles2
# from text_processing_functions import process_all_articles
# from text_processing_functions import remove_stopwords
# from text_processing_functions import get_simple_corpus_dictionary_bow
# from text_processing_functions import entity_recognizer
# from text_processing_functions import get_topic_words_mean_std_prob_frequency
# from text_processing_functions import sort_topics_mean_frequency

import pickle
print('DONE')

ModuleNotFoundError: No module named 'debiaser_validation_function'

In [13]:
return_suggested_articles2(url_main[0])

/Users/sagarsetru/anaconda3/envs/debiaser/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AFTER STOPWORDS
[['blocked'], ['cruise', 'ships', 'docked'], [], ['sail', 'extended', 'upset', 'tourism', 'crucial', 'swing', 'florida', 'blocked', 'centers', 'disease', 'prevention', 'cruise', 'ships', 'docked', 'mid', 'displeased', 'politically', 'powerful', 'tourism', 'crucial', 'swing', 'florida', 'sail', 'originally', 'extended', 'expire'], ['dr', 'robert', 'redfield', 'recommended', 'extension', 'worried', 'cruise', 'ships', 'viral', 'hot', 'spots', 'beginning', 'pandemic', 'coronavirus', 'task', 'dr', 'redfield', 'overruled', 'authorized', 'spoke', 'condition', 'anonymity'], ['ships', 'sail', 'oct', 'agreed', 'voluntary'], ['rejection', 'original', 'axios', 'dr'], ['redfield', 'scolded', 'promoting', 'mask', 'wearing', 'cautioning', 'vaccines', 'widely', 'worried', 'fired', 'considered', 'resigning', 'required', 'oversee', 'compromised', 'dr', 'redfield', 'brian', 'morgenstern', 'deputy', 'cruise', 'ship', 'politically', 'motivated'], ['vice', 'task', 'science', 'implement', 'po

([(0,
   [('cruise', 0.027821178),
    ('dr', 0.023184292),
    ('redfield', 0.01545616),
    ('sail', 0.01545616),
    ('florida', 0.013910533),
    ('virus', 0.012364908),
    ('ship', 0.012364906),
    ('coronavirus', 0.012364903),
    ('ships', 0.010819281),
    ('passengers', 0.009273655)])],
 [[(0, 1)],
  [(1, 1), (2, 1), (3, 1)],
  [],
  [(0, 1),
   (1, 1),
   (2, 1),
   (3, 1),
   (4, 1),
   (5, 2),
   (6, 1),
   (7, 1),
   (8, 1),
   (9, 2),
   (10, 2),
   (11, 1),
   (12, 1),
   (13, 1),
   (14, 1),
   (15, 1),
   (16, 2),
   (17, 2),
   (18, 2),
   (19, 1)],
  [(1, 1),
   (3, 1),
   (20, 1),
   (21, 1),
   (22, 1),
   (23, 1),
   (24, 1),
   (25, 2),
   (26, 1),
   (27, 1),
   (28, 1),
   (29, 1),
   (30, 1),
   (31, 2),
   (32, 1),
   (33, 1),
   (34, 1),
   (35, 1),
   (36, 1),
   (37, 1)],
  [(3, 1), (16, 1), (38, 1), (39, 1), (40, 1)],
  [(25, 1), (41, 1), (42, 1), (43, 1)],
  [(1, 1),
   (13, 1),
   (25, 1),
   (31, 2),
   (37, 1),
   (44, 1),
   (45, 1),
   (46, 1),
  

In [2]:
# with 5 words

# list of main urls
url_main = [
    'https://www.nytimes.com/2020/09/30/health/covid-cruise-ships.html'
]

urls_queried = [
    'https://abcnews.go.com/Health/wireStory/crew-member-cruise-ship-virus-cases-dies-florida-70073196',
    'https://abcnews.go.com/International/wireStory/german-cruise-ship-sets-sail-hopes-short-trip-71986996',
    'https://apnews.com/article/dbc1f2f76ec33ad06d0a78c4ca3e774a',
    'https://apnews.com/article/ap-top-news-travel-international-news-changing-economy-travel-e9878e281e71d58a6bd4e7f88a24797d',
    'https://www.breitbart.com/news/german-cruise-ship-sets-sail-hopes-short-trip-thwarts-virus/',
    'https://www.forbes.com/sites/alexledsom/2020/09/30/white-house-blocks-cdcs-idea-to-extend-no-sail-order-until-feb-2021/#3e2381e73e42',
    'https://www.foxnews.com/travel/cdc-extends-no-sail-order-cruise-ships-coronavirus-crisis',
    'https://www.foxnews.com/travel/zaandam-cruise-ship-fourth-person-coronavirus-dies',
    'https://www.theguardian.com/us-news/2020/oct/01/florida-keys-cruise-ship-ban-covid-19',
    'https://thehill.com/policy/transportation/518975-white-house-blocked-extending-no-sail-cruise-ship-order-until-february',
    'https://thehill.com/policy/healthcare/486115-second-princess-cruise-ship-with-possible-coronavirus-onboard-held-offshore',
    'https://www.nbcnews.com/health/health-news/live-blog/2020-04-09-coronavirus-news-n1179786/ncrd1180841#blogHeader',
    'https://www.nytimes.com/2020/09/30/health/covid-cruise-ships.html',
    'https://www.nytimes.com/2020/09/30/briefing/presidential-debate-cruise-ships-nba.html',
    'https://www.newsmax.com/world/globaltalk/germany-virus-outbreak-cruises/2020/07/25/id/979000/',
    'https://www.npr.org/sections/coronavirus-live-updates/2020/10/01/919082224/the-cdc-extends-no-sail-order-for-cruise-lines',
    'https://www.npr.org/sections/coronavirus-live-updates/',
    'https://reason.com/video/how-china-corrupted-the-world-health-organizations-response-to-covid-19/',
]

urls_random = [
    'https://abcnews.go.com/Health/wireStory/australia-expects-covid-19-vaccination-year-73467063',
    'https://abcnews.go.com/Politics/wireStory/questions-pence-harris-prepare-debate-faceoff-73458831',
    'https://apnews.com/article/election-2020-virus-outbreak-joe-biden-donald-trump-campaigns-3e201de86931e092efb31de0284ed67e',
    'https://apnews.com/article/virus-outbreak-india-archive-economy-a6b30f224b37015e982fd557dcf522d5',
    'https://www.breitbart.com/economy/2020/10/06/pelosi-killed-the-bill/',
    'https://www.forbes.com/sites/lanabandoim/2020/10/06/tally-the-cute-shelf-scanning-robot-is-coming-to-more-grocery-stores/#6105129e177f',
    'https://www.foxnews.com/sports/patriots-james-white-fathers-tragic-death-meant-whole-lot',
    'https://www.foxnews.com/sports/acuna-hr-plunked-again-braves-top-marlins-9-5-to-open-nlds',
    'https://www.theguardian.com/environment/2020/oct/07/one-guy-pulled-a-crowbar-on-me-why-fly-tipping-wars-are-raging-across-britain',
    'https://thehill.com/policy/energy-environment/519909-biden-would-face-hurdles-undoing-trump-environmental-rollbacks',
    'https://thehill.com/homenews/campaign/519851-trump-biden-tied-when-it-comes-to-economy-poll',
    'https://www.nbcnews.com/news/us-news/justice-department-officials-drove-family-separation-policy-watchdog-report-says-n1242375',
    'https://www.nytimes.com/2020/10/07/nyregion/register-to-vote-nyc.html',
    'https://www.nytimes.com/2020/10/06/nyregion/graffiti-artists-5pointz.html',
    'https://www.newsmax.com/politics/stephen-miller-covid-19-testing-positive/2020/10/06/id/990703/',
    'https://www.npr.org/2020/10/07/921055011/trumps-covid-19-treatment-hangs-over-vice-presidential-debate',
    'https://www.npr.org/2020/10/06/920684113/michelle-obama-makes-final-pitch-vote-for-joe-biden-like-your-lives-depend-on-it',
    'https://reason.com/2020/10/06/eddie-van-halen-melting-pot-virtuoso/',
]

# no hits: aljazeera, atlantic, bbc, buzzfeed, cbn, cbs, democracy now, factcheck, huffpost, national review, reuters, salon

# fails for nearly same event: newsmax, wsj

#     https://nypost.com/2020/04/10/cdc-extends-no-sail-order-for-all-cruises-amid-coronavirus-crisis/
#     https://nypost.com/2020/07/16/cdc-bans-us-cruises-through-september-due-to-ongoing-outbreaks-on-ships/

# doesn't work: CNN, daily beast, msnbc, nypost, spectator
# buggy: mother jones, newsmax, politico, salon

In [5]:
print(len(urls_random))
print(len(urls_queried))

18
18
